In [1]:
%load_ext autoreload
%autoreload 2

import os

import warnings
warnings.filterwarnings('ignore')

In [2]:
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from coffea import processor, hist

from processor.FCNC_cutflow import FCNC_cutflow
from Tools.config_helpers import loadConfig
from klepto.archives import dir_archive

lumi = {2016: 35.9, 2017: 41.5, 2018: 59.71}

In [20]:
from processor.default_accumulators import desired_output, add_processes_to_output

from Tools.helpers import get_samples, cutflow_scale_and_merge
from Tools.config_helpers import redirector_ucsd, redirector_fnal
from Tools.nano_mapping import make_fileset, nano_mapping

from processor.meta_processor import get_sample_meta

overwrite = True
local = True

# load the config and the cache
cfg = loadConfig()

cacheName = 'dielectron_mass'
cache = dir_archive(os.path.join(os.path.expandvars(cfg['caches']['base']), cacheName), serialized=True)

year = 2018

# get a python dictionary of all NanoAOD samples
# The samples definitions can be found in data/samples.yaml
samples = get_samples(year)

# make a fileset, taking the definitions in Tools/nano_mapping.py
nano_mappings = nano_mapping(year)
fileset = make_fileset(['Fakes_Flips', 'Rares', 'hct', 'hut'], year, redirector=redirector_ucsd, small=False)
meta = get_sample_meta(fileset, samples)
# 'DY', 'Rares', 'TT/TW', 'TTH', 'TTW', 'TTZ', 'TTVV', 'W', 'X+g'
# in order for cutflows to work we need to add every process to the output accumulator
add_processes_to_output(fileset, desired_output)

histograms = sorted(list(desired_output.keys()))

if local:

    exe_args = {
        'workers': 16,
        'function_args': {'flatten': False},
        "schema": NanoAODSchema,
        "skipbadfiles": True,
    }
    exe = processor.futures_executor

else:
    from Tools.helpers import get_scheduler_address
    from dask.distributed import Client, progress

    scheduler_address = get_scheduler_address()
    c = Client(scheduler_address)

    exe_args = {
        'client': c,
        'function_args': {'flatten': False},
        "schema": NanoAODSchema,
        "skipbadfiles": True,
    }
    exe = processor.dask_executor


if not overwrite:
    cache.load()

if cfg == cache.get('cfg') and histograms == cache.get('histograms') and cache.get('simple_output'):
    output = cache.get('simple_output')
    
else:
    print ("I'm running now")

    output = processor.run_uproot_job(
        fileset,
        "Events",
        FCNC_cutflow(year=year, variations=[], accumulator=desired_output),
        exe,
        exe_args,
        chunksize=250000,
    )

    cache['fileset']        = fileset
    cache['cfg']            = cfg
    cache['histograms']     = histograms
    cache['simple_output']  = output
    cache.dump()

Processing:   0%|          | 0/1096 [00:00<?, ?chunk/s]

I'm running now


Processing:   0%|          | 0/3043 [00:00<?, ?chunk/s]

In [ ]:
print(output['totalEvents']['all']/1e6)

In [22]:
if year == 2016:
    output['hct'] = output['/TT_FCNC-TtoHJ_aTleptonic_HToWWZZtautau_eta_hct-MadGraph5-pythia8/RunIISummer16NanoAODv7-PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/NANOAODSIM'] + output['/TT_FCNC-aTtoHJ_Tleptonic_HToWWZZtautau_eta_hct-MadGraph5-pythia8/RunIISummer16NanoAODv7-PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/NANOAODSIM']
    output['hut'] = output['/TT_FCNC-TtoHJ_aTleptonic_HToWWZZtautau_eta_hut-MadGraph5-pythia8/RunIISummer16NanoAODv7-PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/NANOAODSIM'] + output['/TT_FCNC-aTtoHJ_Tleptonic_HToWWZZtautau_eta_hut-MadGraph5-pythia8/RunIISummer16NanoAODv7-PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/NANOAODSIM']

if year == 2017:
    output['hct'] = output['/TT_FCNC-TtoHJ_aTleptonic_HToWWZZtautau_eta_hct_TuneCP5-MadGraph5-pythia8/RunIIFall17NanoAODv7-PU2017_12Apr2018_Nano02Apr2020_tauDecays_102X_mc2017_realistic_v8-v1/NANOAODSIM'] + output['/TT_FCNC-aTtoHJ_Tleptonic_HToWWZZtautau_eta_hct_TuneCP5-MadGraph5-pythia8/RunIIFall17NanoAODv7-PU2017_12Apr2018_Nano02Apr2020_tauDecays_102X_mc2017_realistic_v8-v1/NANOAODSIM']
    output['hut'] = output['/TT_FCNC-TtoHJ_aTleptonic_HToWWZZtautau_eta_hut_TuneCP5-MadGraph5-pythia8/RunIIFall17NanoAODv7-PU2017_12Apr2018_Nano02Apr2020_tauDecays_102X_mc2017_realistic_v8-v1/NANOAODSIM'] + output['/TT_FCNC-aTtoHJ_Tleptonic_HToWWZZtautau_eta_hut_TuneCP5-MadGraph5-pythia8/RunIIFall17NanoAODv7-PU2017_12Apr2018_Nano02Apr2020_tauDecays_102X_mc2017_realistic_v8-v1/NANOAODSIM'] 

if year == 2018:
    output['hct'] = output['/TT_FCNC-TtoHJ_aTleptonic_HToWWZZtautau_eta_hct_TuneCP5-MadGraph5-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_tauDecays_102X_upgrade2018_realistic_v21-v1/NANOAODSIM'] + output['/TT_FCNC-aTtoHJ_Tleptonic_HToWWZZtautau_eta_hct_TuneCP5-MadGraph5-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_tauDecays_102X_upgrade2018_realistic_v21-v1/NANOAODSIM'] #+ output['/ST_FCNC-TH_Tleptonic_HToWWZZtautau_eta_hct-MadGraph5-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_tauDecays_102X_upgrade2018_realistic_v21-v1/NANOAODSIM']
    output['hut'] = output['/TT_FCNC-TtoHJ_aTleptonic_HToWWZZtautau_eta_hut_TuneCP5-MadGraph5-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_tauDecays_102X_upgrade2018_realistic_v21-v1/NANOAODSIM'] + output['/TT_FCNC-aTtoHJ_Tleptonic_HToWWZZtautau_eta_hut_TuneCP5-MadGraph5-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_tauDecays_102X_upgrade2018_realistic_v21-v1/NANOAODSIM'] #+ output['/ST_FCNC-TH_Tleptonic_HToWWZZtautau_eta_hut-MadGraph5-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_tauDecays_102X_upgrade2018_realistic_v21-v1/NANOAODSIM']

In [23]:
from Tools.helpers import getCutFlowTable, cutflow_scale_and_merge

lines = ['entry', '"skim"', 'lepton selection', 'leading lepton', 'triggers', 'filter', 'ss', 'SS onZ veto', 'two jets', 'MET > 50']
#lines2 = ['entry', '"skim"', 'lepton selection', 'leading lepton', 'triggers', 'filter', 'multilep', 'one jet', 'MET > 50']

output2 = cutflow_scale_and_merge(output, meta, fileset, nano_mappings, lumi=lumi[year])
df = getCutFlowTable(output, processes=['hct', 'hut'], absolute=True, lines=lines, significantFigures=7, signal=['hct', 'hut'])
df

,hct,hut,S/B
entry,1956689.0 +/- 1399.0,1924814.0 +/- 1387.0,1
"""skim""",108100.0 +/- 328.8,106502.0 +/- 326.3,1
lepton selection,77597.0 +/- 278.56,76513.0 +/- 276.61,1
leading lepton,76950.0 +/- 277.4,75897.0 +/- 275.49,1
triggers,71187.0 +/- 266.81,70102.0 +/- 264.77,1
filter,71152.0 +/- 266.74,70069.0 +/- 264.71,1
ss,61181.0 +/- 247.35,60097.0 +/- 245.15,1
SS onZ veto,58655.0 +/- 242.19,57615.0 +/- 240.03,1
two jets,47472.0 +/- 217.88,47449.0 +/- 217.83,1
MET > 50,31621.0 +/- 177.82,31834.0 +/- 178.42,1


In [ ]:
import gzip
import pickle
import cloudpickle
outname = 'backgrounds_and_signal_ttHiD_'+str(year)
os.system("mkdir -p histos/")
print('Saving output in %s...'%("histos/" + outname + ".pkl.gz"))
with gzip.open("histos/" + outname + ".pkl.gz", "wb") as fout:
    cloudpickle.dump(output, fout)
print('Done!')

In [24]:
# import the plotting libararies: matplotlib and mplhep

import matplotlib.pyplot as plt
import mplhep as hep
plt.style.use(hep.style.CMS)

import numpy as np

# load the functions to make a nice plot from the output histograms
# and the scale_and_merge function that scales the individual histograms
# to match the physical cross section

from plots.helpers import scale_and_merge

# define a few axes that we can use to rebin our output histograms

N_bins         = hist.Bin('multiplicity', r'$N$', 10, -0.5, 9.5)
N_bins_red     = hist.Bin('multiplicity', r'$N$', 2, -0.5, 0.5)
pt_bins        = hist.Bin('pt', r'$p_{T}\ (GeV)$', np.array([15, 40, 60, 80, 100, 200, 300]))
pt_fine_bins   = hist.Bin('pt', r'$p_{T}\ (GeV)$', 300, 0, 300)
pt_rebin       = hist.Bin('pt', r'$p_{T}\ (GeV)$', 100, 0, 500)
pt_rebin2      = hist.Bin('pt', r'$p_{T}\ (GeV)$', 60, 0, 300)
pt_rebin22     = hist.Bin('pt', r'$p_{T}\ (GeV)$', 100, 0, 3000)
eta_bins       = hist.Bin('eta', r'$\eta $', np.array([0, 0.8, 1.479, 2.5]))
eta_rebin      = hist.Bin('eta', r'$\eta $', 25, -2.5, 2.5) 
eta_rebin2      = hist.Bin('eta', r'$\eta $', 50, -5, 5)    
phi_bins       = hist.Bin('phi', r'$\phi $', 16, -3.2, 3.2)
mass_bins      = hist.Bin('mass', r'$mass (GeV/c^2)$', 60, 0, 300)

# 1D Histograms

In [25]:
from yahist import Hist1D, Hist2D

In [26]:
def get_total(histos, keys):
        tmp = Hist1D.from_bincounts(np.zeros(len(histos[keys[0]].counts)), histos[keys[0]].edges, )
        for key in keys:
            tmp += histos[key]
        return tmp

def add_uncertainty(hist, ax, ratio=False):
    opts = {'step': 'post', 'label': 'Uncertainty', 'hatch': '///',
                    'facecolor': 'none', 'edgecolor': (0, 0, 0, .5), 'linewidth': 0, 'zorder':10.}
    
    if ratio:
        down = np.ones(len(hist.counts)) - hist.errors/hist.counts
        up = np.ones(len(hist.counts)) + hist.errors/hist.counts
    else:
        down = hist.counts-hist.errors
        up = hist.counts+hist.errors
    ax.fill_between(x=hist.edges, y1=np.r_[down, down[-1]], y2=np.r_[up, up[-1]], **opts)

In [27]:
def f(c, ce):
    out = "{:2g} \n $\pm${:.2f}".format(c, ce)
    return out

# Limits

## Yields

In [47]:
tmp1 = scale_and_merge(output['j_vs_b_ss'], meta, fileset, nano_mappings, lumi=lumi[year])
tmp2 = scale_and_merge(output['j_vs_b_ss_flips'], meta, fileset, nano_mappings, lumi=lumi[year])
tmp3 = scale_and_merge(output['j_vs_b_ss_fakes'], meta, fileset, nano_mappings, lumi=lumi[year])
tmp4 = scale_and_merge(output['j_vs_b_ss_non_fakes_flips'], meta, fileset, nano_mappings, lumi=lumi[year])


h1_hut = Hist2D.from_bincounts(
    tmp1.values(overflow = 'over')[('hut',)].T,
    (tmp1.axis('n1').edges(overflow = 'over'), tmp1.axis('n2').edges(overflow = 'over')),
    errors = np.sqrt(tmp1.values(sumw2=True, overflow = 'over')[('hut',)][1].T),
)

h1_hut = h1_hut*0.01

h1_hct = Hist2D.from_bincounts(
    tmp1.values(overflow = 'over')[('hct',)].T,
    (tmp1.axis('n1').edges(overflow = 'over'), tmp1.axis('n2').edges(overflow = 'over')),
    errors = np.sqrt(tmp1.values(sumw2=True, overflow = 'over')[('hct',)][1].T),
)

h1_hct = h1_hct*0.01

h1_rare = Hist2D.from_bincounts(
    tmp1.values(overflow = 'over')[('Rares',)].T,
    (tmp1.axis('n1').edges(overflow = 'over'), tmp1.axis('n2').edges(overflow = 'over')),
    errors = np.sqrt(tmp1.values(sumw2=True, overflow = 'over')[('Rares',)][1].T),
)

h1_flip = Hist2D.from_bincounts(
    tmp2.values(overflow = 'over')[('Fakes_Flips',)].T,
    (tmp2.axis('n1').edges(overflow = 'over'), tmp2.axis('n2').edges(overflow = 'over')),
    errors = np.sqrt(tmp2.values(sumw2=True, overflow = 'over')[('Fakes_Flips',)][1].T),
)

h1_fake = Hist2D.from_bincounts(
    tmp3.values(overflow = 'over')[('Fakes_Flips',)].T,
    (tmp3.axis('n1').edges(overflow = 'over'), tmp3.axis('n2').edges(overflow = 'over')),
    errors = np.sqrt(tmp3.values(sumw2=True, overflow = 'over')[('Fakes_Flips',)][1].T),
)

h1_non_fake_flip = Hist2D.from_bincounts(
    tmp4.values(overflow = 'over')[('Fakes_Flips',)].T,
    (tmp4.axis('n1').edges(overflow = 'over'), tmp4.axis('n2').edges(overflow = 'over')),
    errors = np.sqrt(tmp4.values(sumw2=True, overflow = 'over')[('Fakes_Flips',)][1].T),
)

h1_non_fake_flip_r = Hist2D.from_bincounts(
    tmp4.values(overflow = 'over')[('Rares',)].T,
    (tmp4.axis('n1').edges(overflow = 'over'), tmp4.axis('n2').edges(overflow = 'over')),
    errors = np.sqrt(tmp4.values(sumw2=True, overflow = 'over')[('Rares',)][1].T),
)

In [ ]:
fig, ax  = plt.subplots(1, 1,figsize=(10,10) )
h1_hct.plot(show_counts=True, equidistant='xy', counts_formatter=f)
ax.set_xlabel(r'$N_{jets}$')
ax.set_ylabel(r'$N_{b}$')

In [ ]:
fig, ax  = plt.subplots(1, 1,figsize=(10,10) )
h1_hut.plot(show_counts=True, equidistant='xy', counts_formatter=f)
ax.set_xlabel(r'$N_{jets}$')
ax.set_ylabel(r'$N_{b}$')

In [ ]:
fig, ax  = plt.subplots(1, 1,figsize=(10,10) )
h1_fake.plot(show_counts=True, equidistant='xy', counts_formatter=f)
ax.set_xlabel(r'$N_{jets}$')
ax.set_ylabel(r'$N_{b}$')

In [42]:
np.sum(h1_fake.counts)

1201.9634041422898

In [ ]:
fig, ax  = plt.subplots(1, 1,figsize=(10,10) )
h1_flip.plot(show_counts=True, equidistant='xy', counts_formatter=f)
ax.set_xlabel(r'$N_{jets}$')
ax.set_ylabel(r'$N_{b}$')

In [50]:
np.sum(h1_flip.counts)

84.72630812654178

In [ ]:
fig, ax  = plt.subplots(1, 1,figsize=(10,10) )
h1_rare.plot(show_counts=True, equidistant='xy', counts_formatter=f)
ax.set_xlabel(r'$N_{jets}$')
ax.set_ylabel(r'$N_{b}$')

In [51]:
np.sum(h1_rare.counts)

1524.6661248688392

In [ ]:
fig, ax  = plt.subplots(1, 1,figsize=(10,10) )
h1_non_fake_flip.plot(show_counts=True, equidistant='xy', counts_formatter=f)
ax.set_xlabel(r'$N_{jets}$')
ax.set_ylabel(r'$N_{b}$')

In [52]:
np.sum(h1_non_fake_flip.counts)

1161.0425518008649

In [ ]:
tmp5 = scale_and_merge(output['j_vs_b_ml'], meta, fileset, nano_mappings, lumi=lumi[year])


h2_hut = Hist2D.from_bincounts(
    tmp5.values(overflow = 'all')[('hut',)].T,
    (tmp5.axis('n1').edges(overflow = 'all'), tmp5.axis('n2').edges(overflow = 'all')),
    errors = np.sqrt(tmp5.values(sumw2=True, overflow = 'all')[('hut',)][1].T),
)

h2_hut = h2_hut*0.01

h2_hct = Hist2D.from_bincounts(
    tmp5.values(overflow = 'all')[('hct',)] .T,
    (tmp5.axis('n1').edges(overflow = 'all'), tmp5.axis('n2').edges(overflow = 'all')),
    errors = np.sqrt(tmp5.values(sumw2=True, overflow = 'all')[('hct',)][1].T),
)

h2_hct = h2_hct*0.01

h2_fake = Hist2D.from_bincounts(
    tmp2.values(overflow = 'all')[('Flips',)].T,
    (tmp2.axis('n1').edges(overflow = 'all'), tmp2.axis('n2').edges(overflow = 'all')),
    errors = np.sqrt(tmp2.values(sumw2=True, overflow = 'all')[('Flips',)][1].T),
)

h2_flip_bins = [np.array([0.5, 1.5, 2.5, 3.5, 4.5]),
                np.array([-1.5, -0.5, 0.5, 1.5, 2.5]),]
            
h2_flip_counts = np.array([[0.01, 0.01, 0.01, 0.01],
                           [0.01, 0.01, 0.01, 0.01],
                           [0.01, 0.01, 0.01, 0.01],
                           [0.01, 0.01, 0.01, 0.01],])

h2_flip_errors = np.array([[0.01, 0.01, 0.01, 0.01],
                           [0.01, 0.01, 0.01, 0.01],
                           [0.01, 0.01, 0.01, 0.01],
                           [0.01, 0.01, 0.01, 0.01],])
            
h2_flip = Hist2D.from_bincounts(h2_flip_counts, h2_flip_bins, h2_flip_errors)

h2_rare = Hist2D.from_bincounts(
    tmp2.values(overflow = 'all')[('Rares',)].T,
    (tmp2.axis('n1').edges(overflow = 'all'), tmp2.axis('n2').edges(overflow = 'all')),
    errors = np.sqrt(tmp2.values(sumw2=True, overflow = 'all')[('Rares',)][1].T),
)

In [ ]:
fig, ax  = plt.subplots(1, 1,figsize=(10,10) )
h2_hct.plot(counts=True, equidistant='xy', counts_formatter=f)
ax.set_xlabel(r'$N_{jets}$')
ax.set_ylabel(r'$N_{b}$')

In [ ]:
fig, ax  = plt.subplots(1, 1,figsize=(10,10) )
h2_hut.plot(counts=True, equidistant='xy', counts_formatter=f)
ax.set_xlabel(r'$N_{jets}$')
ax.set_ylabel(r'$N_{b}$')

In [ ]:
fig, ax  = plt.subplots(1, 1,figsize=(10,10) )
h2_fake.plot(counts=True, equidistant='xy', counts_formatter=f)
ax.set_xlabel(r'$N_{jets}$')
ax.set_ylabel(r'$N_{b}$')

In [ ]:
fig, ax  = plt.subplots(1, 1,figsize=(10,10) )
h2_flip.plot(counts=True, equidistant='xy', counts_formatter=f)
ax.set_xlabel(r'$N_{jets}$')
ax.set_ylabel(r'$N_{b}$')

In [ ]:
fig, ax  = plt.subplots(1, 1,figsize=(10,10) )
h2_rare.plot(counts=True, equidistant='xy', counts_formatter=f)
ax.set_xlabel(r'$N_{jets}$')
ax.set_ylabel(r'$N_{b}$')

## Datacards

In [ ]:
from Tools.dataCard import dataCard

c = dataCard()
c.setPrecision(3)

c.addUncertainty('fakerate',    'lnN')
c.addUncertainty('fliprate',    'lnN')
c.addUncertainty('rare_norm',   'lnN')
c.addUncertainty('signal_norm', 'lnN')
c.addUncertainty('lumi',        'lnN')

binnum = 0


for b in range(0,3):
    for j in range(0,3):
        binname = 'bin'+str(binnum)
        Binname = 'DL_'+str(b)+'_'+str(j+2)
        binnum += 1
        c.addBin(binname, ['fake', 'flip', 'rare'], Binname) # signal is automatically added
        
        processes = {'signal': h1_hct, 'fake': h1_fake, 'flip': h1_flip, 'rare': h1_rare}
        for process in processes:
            uname = 'Stat_'+binname+'_'+process
            c.addUncertainty(uname, 'lnN')
            c.specifyUncertainty(uname, binname, process, round(1+processes[process].errors[b][j]/processes[process].counts[b][j], 3))
        
        c.specifyExpectation(binname, 'signal',  round(h1_hct.counts[b][j], 3))
        c.specifyExpectation(binname, 'fake',    round(h1_fake.counts[b][j], 3))
        c.specifyExpectation(binname, 'flip',    round(h1_flip.counts[b][j], 3))
        c.specifyExpectation(binname, 'rare',    round(h1_rare.counts[b][j], 3))

        c.specifyUncertainty('signal_norm', binname, 'signal', 1.01)
        c.specifyUncertainty('fakerate',    binname, 'fake',   1.40)
        c.specifyUncertainty('fliprate',    binname, 'flip',   1.30)
        c.specifyUncertainty('rare_norm',   binname, 'rare',   1.30)

        c.specifyObservation(binname, round(h1_fake.counts[b][j] + h1_flip.counts[b][j] + h1_rare.counts[b][j], 3))
        
for b in range(1,4):
    for j in range(0,4):
        binname = 'bin'+str(binnum)
        Binname = 'ML_'+str(b)+'_'+str(j+1)
        binnum += 1
        
        c.addBin(binname, ['fake', 'flip', 'rare'], Binname) # signal is automatically added
        
        processes = {'signal': h2_hct, 'fake': h2_fake, 'flip': h2_flip, 'rare': h2_rare}
        for process in processes:
            uname = 'Stat_'+binname+'_'+process
            c.addUncertainty(uname, 'lnN') 
            c.specifyUncertainty(uname, binname, process, round(1+processes[process].errors[b][j]/processes[process].counts[b][j], 3))
        
        c.specifyExpectation(binname, 'signal',  round(h2_hct.counts[b][j], 3))
        c.specifyExpectation(binname, 'fake',    round(h2_fake.counts[b][j], 3))
        c.specifyExpectation(binname, 'flip',    round(h2_flip.counts[b][j], 3))
        c.specifyExpectation(binname, 'rare',    round(h2_rare.counts[b][j], 3))

        c.specifyUncertainty('signal_norm', binname, 'signal', 1.01)
        c.specifyUncertainty('fakerate',    binname, 'fake',   1.40)
        c.specifyUncertainty('fliprate',    binname, 'flip',   1.30)
        c.specifyUncertainty('rare_norm',   binname, 'rare',   1.30)

        c.specifyObservation(binname, round(h2_fake.counts[b][j] + h2_flip.counts[b][j] + h2_rare.counts[b][j], 3))
    
c.specifyFlatUncertainty('lumi', 1.02)


c.writeToFile('./FCNC_hct_ttH_'+str(year)+'_1.txt')


#res = c.calcLimit('./FCNC_hct_ttH_'+str(year)+'_1.txt')

In [ ]:
from Tools.dataCard import dataCard

c = dataCard()
c.setPrecision(3)

c.addUncertainty('fakerate', 'lnN')
c.addUncertainty('fliprate', 'lnN')
c.addUncertainty('rare_norm', 'lnN')
c.addUncertainty('signal_norm', 'lnN')
c.addUncertainty('lumi', 'lnN')

binnum = 0

for b in range(0,3):
    for j in range(0,3):
        binname = 'bin'+str(binnum)
        Binname = 'DL_'+str(b)+'_'+str(j+2)
        binnum += 1
        c.addBin(binname, ['fake', 'flip', 'rare'], Binname) # signal is automatically added
        
        processes = {'signal': h1_hut, 'fake': h1_fake, 'flip': h1_flip, 'rare': h1_rare}
        for process in processes:
            uname = 'Stat_'+binname+'_'+process
            c.addUncertainty(uname, 'lnN')
            c.specifyUncertainty(uname, binname, process, 1+processes[process].errors[b][j]/processes[process].counts[b][j])
        
        c.specifyExpectation(binname, 'signal',  h1_hut.counts[b][j])
        c.specifyExpectation(binname, 'fake',    h1_fake.counts[b][j])
        c.specifyExpectation(binname, 'flip',    h1_flip.counts[b][j])
        c.specifyExpectation(binname, 'rare',    h1_rare.counts[b][j])

        c.specifyUncertainty('signal_norm', binname, 'signal', 1.01)
        c.specifyUncertainty('fakerate',    binname, 'fake',   1.40)
        c.specifyUncertainty('fliprate',    binname, 'flip',   1.30)
        c.specifyUncertainty('rare_norm',   binname, 'rare',   1.30)

        c.specifyObservation(binname, h1_fake.counts[b][j]+h1_flip.counts[b][j]+h1_rare.counts[b][j])
        
for b in range(1,4):
    for j in range(0,4):
        binname = 'bin'+str(binnum)
        Binname = 'ML_'+str(b)+'_'+str(j+1)
        binnum += 1
        
        c.addBin(binname, ['fake', 'flip', 'rare'], Binname) # signal is automatically added
        
        processes = {'signal': h2_hut, 'fake': h2_fake, 'flip': h2_flip, 'rare': h2_rare}
        for process in processes:
            uname = 'Stat_'+binname+'_'+process
            c.addUncertainty(uname, 'lnN')
            c.specifyUncertainty(uname, binname, process, round(1+processes[process].errors[b][j]/processes[process].counts[b][j], 3))
        
        c.specifyExpectation(binname, 'signal',  h2_hut.counts[b][j])
        c.specifyExpectation(binname, 'fake',    h2_fake.counts[b][j])
        c.specifyExpectation(binname, 'flip',    h2_flip.counts[b][j])
        c.specifyExpectation(binname, 'rare',    h2_rare.counts[b][j])

        c.specifyUncertainty('signal_norm', binname, 'signal', 1.01)
        c.specifyUncertainty('fakerate',    binname, 'fake',   1.40)
        c.specifyUncertainty('fliprate',    binname, 'flip',   1.30)
        c.specifyUncertainty('rare_norm',   binname, 'rare',   1.30)

        c.specifyObservation(binname, h2_fake.counts[b][j]+h2_flip.counts[b][j]+h2_rare.counts[b][j])
    
c.specifyFlatUncertainty('lumi', 1.02)


c.writeToFile('./FCNC_hut_ttH_'+str(year)+'_1.txt')


#res = c.calcLimit('./FCNC_hut_ttH_'+str(year)+'_1.txt')
